# Word2Vec (Negative Sampling)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
np.__version__, torch.__version__

('1.26.4', '2.5.1')

In [3]:
import matplotlib
matplotlib.__version__

'3.9.2'

## 1. Load data

In [4]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/tadasuttaket/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [5]:
from nltk.corpus import brown

brown.categories()

#returns a list of sentences, where each sentence is a list of words from the "news" category of the Brown Corpus
corpus = brown.sents(categories="news")
corpus

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [6]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

In [7]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
word2index['dog']

8667

In [8]:
last_word = len(vocabs)
last_word

14394

In [9]:
vocabs.append('<UNK>')
word2index['<UNK>'] = last_word

In [10]:
index2word = {v:k for k, v in word2index.items()}
index2word[2035]

'problem'

## 2. Prepare train data

In [11]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus):

    skipgrams = []

    window_size = 2

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            #outside words = 2 words
            outside = [word2index[doc[j]] for j in range(i - window_size, i + window_size + 1) if j != i]
            #for each of these two outside words, we gonna append to a list
            for each_out in outside:
                skipgrams.append([center, each_out])
                #center, outside1;   center, outside2;   center, outside3;   center, outside4
                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, corpus)

In [12]:
x.shape  #batch_size, 1

(2, 1)

In [13]:
x

array([[3802],
       [7215]])

In [14]:
y.shape  #batch_size 1

(2, 1)

In [15]:
y

array([[11216],
       [ 4709]])

## 3. Negative Sampling

### Unigram distribution

$$P(w)=U(w)^{3/4}/Z$$

In [16]:
z = 0.001

In [17]:
#count
from collections import Counter

word_count = Counter(flatten(corpus))
word_count

#get the total number of words
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

100554

In [18]:
vocabs

['abuses',
 'submarines',
 'Virgil',
 'spraying',
 'opposes',
 'sculptures',
 'nature',
 'brief',
 '1935',
 'uncontrolled',
 'gain',
 'worth',
 'began',
 'vast',
 'quoted',
 'Mediterranean',
 'embedded',
 '10-year',
 'adviser',
 'Feb.',
 'enigma',
 'fairway',
 'codes',
 'struggling',
 'appointing',
 'pledge',
 'wavelengths',
 'playoff',
 'seems',
 'history',
 'Reid',
 'star',
 'summed',
 'Mal',
 "taxpayers'",
 'suggestions',
 'Scampini',
 'Equator',
 'bankrupt',
 'fraternity',
 'ballgowns',
 'manuscript',
 'muscular',
 'Question',
 'propeller',
 'Neveh',
 'Friend',
 'repair',
 'amended',
 'confession',
 'protection',
 'assessments',
 'Venturi',
 'possibly',
 'evasive',
 'products',
 'revision',
 'fallout',
 'rain-slick',
 'investigation',
 'Cris',
 'challenged',
 'Mr.',
 'forbidden',
 'Radio',
 'Todd',
 'Gladden',
 'Lauderdale',
 'plus',
 'Jimmie',
 'walloping',
 'appraisal',
 'deserves',
 'contributions',
 'Brett',
 'Turandot',
 'Tudor',
 'Albert',
 'debuts',
 'Said',
 'developments',

$$P(w)=U(w)^{3/4}/Z$$

In [19]:
unigram_table = []

for v in vocabs:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    unigram_table.extend([v] * uw_alpha)
    
Counter(unigram_table)

Counter({'the': 114,
         ',': 108,
         '.': 89,
         'of': 69,
         'and': 55,
         'to': 55,
         'a': 52,
         'in': 50,
         'for': 30,
         'The': 26,
         'that': 26,
         'is': 24,
         'was': 24,
         "''": 24,
         '``': 24,
         'on': 22,
         'at': 21,
         'be': 19,
         'with': 19,
         'by': 18,
         'as': 18,
         'he': 17,
         'said': 15,
         'his': 15,
         'will': 15,
         'from': 14,
         'it': 14,
         ';': 13,
         'are': 13,
         'an': 12,
         'had': 12,
         'has': 12,
         '--': 12,
         'were': 11,
         'have': 11,
         'who': 11,
         'this': 11,
         'Mrs.': 11,
         'not': 11,
         'their': 10,
         'would': 10,
         'which': 10,
         'they': 9,
         'He': 9,
         'been': 9,
         'Mr.': 8,
         ')': 8,
         'last': 8,
         'I': 8,
         'its': 8,
         'one': 

## 4. Model

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [20]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return torch.LongTensor(idxs)

In [21]:
import random

def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [22]:
batch_size = 2
x, y = random_batch(batch_size, corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)

In [23]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [24]:
y_tensor[1]

tensor([4709])

In [25]:
neg_samples[1]

tensor([ 3802, 12764,  9380,  6592,  6592])

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [26]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size, word2index):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
        self.word2index        = word2index
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)
        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)
    
    def get_embed(self, word):
        word2index = self.word2index
        
        try:
            index = word2index[word]
        except:
            index = word2index['<UNK>']
            
        word = torch.LongTensor([index])
        
        embed_c = self.embedding_center(word)
        embed_o = self.embedding_outside(word)
        embed   = (embed_c + embed_o) / 2
        
        return embed[0][0].item(), embed[0][1].item()

In [27]:
#test your model
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size, word2index)

In [28]:
loss = model(x_tensor, y_tensor, neg_samples)

In [29]:
loss

tensor(2.1931, grad_fn=<NegBackward0>)

## 5. Training

In [30]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
num_epochs = 100

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss = model(input_tensor, label_tensor, neg_samples)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    #print the loss
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss:2.6f}")

Epoch     10 | Loss: 2.141691
Epoch     20 | Loss: 2.466000
Epoch     30 | Loss: 1.385680
Epoch     40 | Loss: 3.830783
Epoch     50 | Loss: 1.411748
Epoch     60 | Loss: 1.203005
Epoch     70 | Loss: 5.030638
Epoch     80 | Loss: 2.998884
Epoch     90 | Loss: 1.701631
Epoch    100 | Loss: 1.144752


In [32]:
def open_file(path_to_file):
    # Open the file in read mode
    try:
        with open(path_to_file, 'r') as file:
            content = file.readlines()
    except FileNotFoundError:
        print(f"The file {path_to_file} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return content

In [33]:
file_path_semantic = "../Test/word-test.v1-capital.txt"
file_path_syntatic = "../Test/word-test.v1-past-tense.txt"

content_semantic = open_file(file_path_semantic)
content_syntactic = open_file(file_path_syntatic)

semantic = []
syntactic = []

for sent in content_semantic:
    semantic.append(sent.strip())

for sent in content_syntactic:
    syntactic.append(sent.strip())

In [34]:
vector_space = []

for word in vocabs:
    vector_space.append(model.get_embed(word))

vector_space = np.array(vector_space)

In [35]:
#scipy version
from scipy import spatial

def cos_sim(a, b):
    cos_sim = 1 - spatial.distance.cosine(a, b)  #distance = 1 - similarlity, because scipy only gives distance
    return cos_sim

def cos_sim_scores(vector_space, target_vector):
    scores = []
    for each_vect in vector_space:
        each_vect = tuple(each_vect)
        target_vector=tuple(target_vector)
        scores.append(cos_sim(target_vector, each_vect))

    return np.array(scores)

In [36]:
def similarity(model, test_data):
    words = test_data.split(" ")

    embed0 = np.array(model.get_embed(words[0]))
    embed1 = np.array(model.get_embed(words[1]))
    embed2 = np.array(model.get_embed(words[2]))

    similar_vector = embed1 - embed0 + embed2

    similarity_scores = cos_sim_scores(vector_space, similar_vector)
    max_score_idx = np.argmax(similarity_scores)
    similar_word = index2word[max_score_idx]

    result = False
    if similar_word == words[3]:
        result = True

    return result

## Semantic accuracy

In [37]:
sem_total = len(semantic)
sem_correct = 0
for sent in semantic:
    if similarity(model, sent):
        sem_correct += 1

In [38]:
sem_accuracy = sem_correct / sem_total
print(f"Semantic accuracy: {sem_accuracy:2.2f}")

Semantic accuracy: 0.00


## Syntactixc

In [39]:
syn_total = len(syntactic)
syn_correct = 0
for sent in syntactic:
    if similarity(model, sent):
        syn_correct += 1

In [40]:
syn_accuracy = syn_correct / syn_total
print(f"Syntactic accuracy: {syn_accuracy:2.2f}")

Syntactic accuracy: 0.00


## Similarity Accuracy

In [41]:
file_path = "../Test/wordsim_similarity_goldstandard.txt"

content = open_file(file_path)

sim_data = []

for sent in content:
    sim_data.append(sent.strip())

In [42]:
def compute_similarity(model, test_data):
    words = test_data.split("\t")

    embed0 = np.array(model.get_embed(words[0].strip()))
    embed1 = np.array(model.get_embed(words[1].strip()))

    similarity_model = embed1 @ embed0.T
    similarity_provided = float(words[2].strip())

    return similarity_provided, similarity_model

In [43]:
ds_scores = []
model_scores = []
for sent in sim_data:
    ds_score, model_score = compute_similarity(model, sent)

    ds_scores.append(ds_score)
    model_scores.append(model_score)

In [44]:
from scipy.stats import spearmanr

corr = spearmanr(ds_scores, model_scores)[0]

print(f"Correlation between the dataset metrics and model scores is {corr:2.2f}.")

Correlation between the dataset metrics and model scores is 0.19.
